In [2]:
# Import libraries

# Pandas
import pandas as pd
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import confusion_matrix


from sklearn.linear_model import LogisticRegression

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import classification_report
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.svm import SVC
import sklearn as skl
import tensorflow as tf

In [4]:
# create DataFrame from the clean csv file
file = 'clean_ml_data_0625.csv'
clean_df = pd.read_csv(file)

In [5]:
# Get a sample of 50,000 rows of data
clean_df = clean_df.sample(50000)

In [6]:
# Check dataset
clean_df.head()

,gender,pneumonia,pregnant,diabetes,copd,asthma,immunosup,hypertension,cardiovascular,obesity,renal_chronic,tobacco,closed_contact,another_complication,death,intubation,ICU,new_age
311375,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5
396244,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
616632,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
960166,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,4
913966,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2


In [7]:
# Check the class distribution
Counter(clean_df.death)

Counter({0: 44936, 1: 5064})

In [8]:
# get the target column ['death'] and rest of the features, excluing death, ICU and Intubation
y= clean_df['death']
X = clean_df.drop(columns = ['ICU','intubation','death'])

In [9]:
# Split the training and test dataset to fit the model
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state =42, stratify=y)

In [10]:
# Creating StandardScaler instance
scaler = StandardScaler()
# Fitting standard scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled_s, y_resampled_s = SMOTE(random_state=1,
sampling_strategy='auto').fit_resample(X_train_scaled,y_train)
Counter(y_resampled_s)

Counter({0: 33702, 1: 33702})

In [ ]:
# from imblearn.under_sampling import RandomUnderSampler
# ros = RandomUnderSampler(random_state=1)
# X_resampled_s, y_resampled_s = ros.fit_resample(X_train_scaled, y_train)
# Counter(y_resampled_s)

In [ ]:
# # Use SMOTEENN resampling method
# from imblearn.combine import SMOTEENN 
# smote_enn = SMOTEENN(random_state=0)
# X_resampled_s, y_resampled_s = smote_enn.fit_resample(X_train_scaled,y_train)
# Counter(y_resampled_s)

In [ ]:
# keep_col = ['gender',
#          'pneumonia', 'new_age',
#        'pregnant', 'diabetes', 'copd',
#        'asthma', 'immunosup', 'hypertension',
#        'cardiovascular', 'obesity', 'renal_chronic', 'tobacco', 'another_complication', 'closed_contact']

In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_of_features = len(X_resampled_s[0])
first_layer_unit = 80
second_layer_unit = 30
first_activation = 'relu'
second_activation = 'relu'

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units = first_layer_unit,input_dim = number_of_features,activation = first_activation))

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units = second_layer_unit,activation = second_activation))

# Output layer
nn.add(
    tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                1280      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 3,741
Trainable params: 3,741
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["Accuracy","TruePositives",'FalseNegatives','Recall'])

In [12]:
# Train the model
fit_model = nn.fit(X_resampled_s, y_resampled_s,epochs = 50)

Epoch 1/50
2109/2109 [==============================] - 9s 3ms/step - loss: 0.3296 - Accuracy: 0.8654 - true_positives: 30153.0000 - false_negatives: 3583.0000 - recall: 0.8938
Epoch 2/50
2109/2109 [==============================] - 5s 2ms/step - loss: 0.3166 - Accuracy: 0.8706 - true_positives: 30316.0000 - false_negatives: 3420.0000 - recall: 0.8986
Epoch 3/50
2109/2109 [==============================] - 5s 3ms/step - loss: 0.3137 - Accuracy: 0.8714 - true_positives: 30332.0000 - false_negatives: 3404.0000 - recall: 0.8991
Epoch 4/50
2109/2109 [==============================] - 6s 3ms/step - loss: 0.3121 - Accuracy: 0.8726 - true_positives: 30390.0000 - false_negatives: 3346.0000 - recall: 0.9008
Epoch 5/50
2109/2109 [==============================] - 8s 4ms/step - loss: 0.3101 - Accuracy: 0.8740 - true_positives: 30413.0000 - false_negatives: 3323.0000 - recall: 0.9015
Epoch 6/50
2109/2109 [==============================] - 8s 4ms/step - loss: 0.3089 - Accuracy: 0.8747 - true_positi

In [13]:
# Evaluate the model using the test data : Recall: tp/(tp+fn) 0.829236
results = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(results)
# print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

391/391 - 2s - loss: 0.3628 - Accuracy: 0.8501 - true_positives: 1069.0000 - false_negatives: 186.0000 - recall: 0.8518 - 2s/epoch - 6ms/step
[0.3627903461456299, 0.8500800132751465, 1069.0, 186.0, 0.8517928123474121]


In [ ]:
# # # Create a method that creates a new Sequential model with hyperparameter options
# def create_model(hp):
#     nn_model = tf.keras.models.Sequential()

#     # Allow kerastuner to decide which activation function to use in hidden layers
#     activation = hp.Choice('activation',['relu','tanh','sigmoid'])
#     first_units = hp.Int('first_units', min_value=1, max_value=10, step=2)
    
#     # Allow kerastuner to decide number of neurons in first layer
#     nn_model.add(tf.keras.layers.Dense(units=first_units, activation=activation, input_dim=len(X_train_scaled[0])))

#     # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
#     for i in range(hp.Int('num_layers', 1, 6)):
#         next_units = hp.Int('units_' + str(i), min_value=1, max_value=10, step=2)

#         nn_model.add(tf.keras.layers.Dense(units=next_units, activation=activation))
    
#     nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

#     # Compile the model
#     nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["Accuracy"])

#     return nn_model

In [ ]:
# import keras_tuner as kt

# tuner = kt.Hyperband(
#     create_model, # function name
#     objective="val_accuracy",
#     max_epochs=20,
#     hyperband_iterations=2)

In [ ]:
# # Run the kerastuner search for best hyperparameters
# tuner.search(X_resampled_s, y_resampled_s,epochs=20,validation_data=(X_test_scaled,y_test))

In [ ]:
# # Get best model hyperparameters
# best_hyper = tuner.get_best_hyperparameters(1)[0]
# best_hyper.values

In [ ]:
# best_model = tuner.get_best_models(1)[0]
# fit_model = best_model.fit(X_resampled_s, y_resampled_s, epochs=20)

In [ ]:
# # Evaluate best model against full test data
# model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
# print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")